In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.3-70b-versatile",groq_api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000183AD991330>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000183AD991240>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Haaris and I am an AI Engineer")])

AIMessage(content="Nice to meet you, Haaris. It's great to hear that you're an AI Engineer. That's a fascinating field, and I'm sure you're doing some exciting work. What specific areas of AI are you interested in or working on? Are you more focused on machine learning, natural language processing, computer vision, or something else?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 48, 'total_tokens': 118, 'completion_time': 0.150793933, 'completion_tokens_details': None, 'prompt_time': 0.004201156, 'prompt_tokens_details': None, 'queue_time': 0.058690093, 'total_time': 0.154995089}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019ba951-b928-77b1-ad9b-081f8a159341-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 48, 'output_tokens': 70, 'total_tokens': 118})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is Haaris and I am an AI Engineer"),
        AIMessage(content="Nice to meet you, Haaris. It's great to hear that you're an AI Engineer. That's a fascinating field, and I'm sure you're doing some exciting work. What specific areas of AI are you interested in or working on? Are you more focused on machine learning, natural language processing, computer vision, or something else?"),
        HumanMessage(content="What is my naem and what do I do?")
    ]
)

AIMessage(content="Your name is Haaris, and you're an AI Engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 138, 'total_tokens': 152, 'completion_time': 0.029863398, 'completion_tokens_details': None, 'prompt_time': 0.020659207, 'prompt_tokens_details': None, 'queue_time': 0.053732533, 'total_time': 0.050522605}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019ba953-f0d0-7aa0-891d-550fbd6f0ee0-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 138, 'output_tokens': 14, 'total_tokens': 152})

## Message History

In [30]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [31]:
config={"configurable":{"session_id":"chat1"}}

In [35]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Haaris and I am an AI Engineer")],
    config=config
)

In [36]:
response.content

"Hi Haaris, welcome. It seems like we're starting fresh again. As an AI Engineer, you're part of a field that's constantly evolving and shaping the future of technology. What inspired you to become an AI Engineer, and what do you enjoy most about your work?"

In [41]:
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config1
)

In [42]:
response.content

"I don't know your name. I'm a large language model, I don't have any information about you, including your name. I'm here to help answer your questions and provide assistance, but I don't have any personal data about you. If you'd like to share your name, I'd be happy to chat with you!"

In [45]:
response=with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)
response.content

'Your name is Haaris.'

## Prompt Template

In [46]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to best of your ability."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain=prompt|model

In [48]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Haaris")]})

AIMessage(content="Hello Haaris, it's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 57, 'total_tokens': 84, 'completion_time': 0.043126725, 'completion_tokens_details': None, 'prompt_time': 0.002949455, 'prompt_tokens_details': None, 'queue_time': 0.053904905, 'total_time': 0.04607618}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bac6b-4d66-7bd3-a656-12038d7e4b0c-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 57, 'output_tokens': 27, 'total_tokens': 84})

In [49]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [51]:
config={"configurable":{"session_id":"chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi my name is Haaris")],
    config=config
)
response.content

"Hello again Haaris, welcome back. I remember you introduced yourself earlier. What's on your mind today? Want to talk about something in particular or just say hello?"

In [52]:
# More complex
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are helpful assistant. Answer all questiojs to best of your ability in {language}."
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [53]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Haaris.")],"language":"Hindi"})
response.content

'नमस्ते हारिस, मैं आपका सहायक हूँ। मैं आपकी किस प्रकार से मदद कर सकता हूँ?'

In [54]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [55]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi, I am Haaris")],"language":"Hindi"},
    config=config,
)
response.content

'नमस्ते हारिस, मैं आपकी मदद करने में खुश हूँ। मैं आपके प्रश्नों का उत्तर देने की पूरी कोशिश करूँगा। क्या आप कुछ पूछना चाहते हैं या किसी विशेष विषय पर चर्चा करना चाहते हैं?'

In [58]:
response=with_message_history.invoke(
    {'messages':[HumanMessage(content="Hi, What is my name?")],"language":"Hindi"},
    config=config,
)
response.content

'आपका नाम हारिस है! मुझे पहले ही पता है, हम पहले भी मिल चुके हैं!'

## Managing the Conversation History

In [62]:
!pip install transformers

In [88]:
from langchain_core.messages import SystemMessage,trim_messages

import tiktoken

encoding = tiktoken.get_encoding("cl100k_base")

def groq_token_counter(messages):
    total = 0
    for m in messages:
        if m.content:
            total += len(encoding.encode(m.content))
    return total

trimmer=trim_messages(
    max_tokens=10,
    strategy="last",
    token_counter=groq_token_counter,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages=[
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi, I am Haaris"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like Vanilla Icecream"),
    AIMessage(content="nice"),
    HumanMessage(content="What is 2+2?"),
    AIMessage(content="4"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={})]

In [89]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)

response=chain.invoke(
    {
        "messages":messages + [HumanMessage(content="Whay ice cream do I like?")],
        "language":"English"
    }
)

response.content

"Thank you for the kind words. I'll do my best to provide helpful and accurate responses to all your questions. Is there anything specific you'd like to know or discuss? I'm here to help."

In [ ]:
from langchain_core.documents import Document

document=